# Notebook ML4RG-Project

First download the data and install the needed packages

In [1]:
![[ ! -d ML4RG-2023-project ]] && git clone https://github.com/Hugenotte585/ML4RG-2023-project.git
!gdown https://drive.google.com/uc?id=16BUHUYXNYvndfsiECB8-C7cwWq82oTg-

Cloning into 'ML4RG-2023-project'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 248 (delta 113), reused 134 (delta 30), pack-reused 0
Receiving objects: 100% (248/248), 2.98 MiB | 17.41 MiB/s, done.
Resolving deltas: 100% (113/113), done.
Downloading...
From: https://drive.google.com/uc?id=16BUHUYXNYvndfsiECB8-C7cwWq82oTg-
To: /content/ML4RG-2023-project.tar
100% 39.8M/39.8M [00:00<00:00, 96.3MB/s]


In [2]:
!tar -xvf ML4RG-2023-project.tar
!rm ML4RG-2023-project.tar

Homo_sapiens_3prime_UTR.fa
Homo_sapiens_3prime_UTR.fa.fai
MLM_mammals_species_aware_5000_weights


In [3]:
!pip -q install pysam
!pip -q install torchmetrics
!pip -q install einops
!pip -q install omegaconf
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.9 MB/s eta 0:00:00


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
#sys.path.insert(0, './ML4RG-2023-project')
sys.path.insert(0, '../..')

import gc
import pysam
import pandas as pd
import re
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np


import helpers.train_eval as train_eval    #train and evaluation
import helpers.misc as misc                #miscellaneous functions

import encoding_utils.sequence_encoders as sequence_encoders
import encoding_utils.sequence_utils as sequence_utils
from models.spec_dss import DSSResNet, DSSResNetEmb, SpecAdd
from models.baseline.markov_model import *
from models.baseline.markov_for_dinuc import *
from Bio import SeqIO
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dinucl = ["AA", "AC", "AT", "AG", "CA", "CC", "CT", "CG", "TA", "TC", "TT", "TG", "GA", "GC", "GG", "GT"]
count_dinuc = dict((el, 0) for el in dinucl)

In [6]:
for record in SeqIO.parse('../../../test/Homo_sapiens_3prime_UTR.fa', 'fasta'):
    for nucleotide in count_dinuc:
        count = 0
        for i in range(len(record.seq)-1):
            pair=record.seq[i:i+2]
            if pair == nucleotide:
                count += 1
        count_dinuc[nucleotide] += count
print('\n'.join(['{}: {}'.format(i,count_dinuc[i]) for i in count_dinuc]))

KeyboardInterrupt: 

In [8]:
s_di = sum(count_dinuc.values())
a2 = {k: v / 4989147 for k, v in count_dinuc.items()}

In [7]:
count_nucletides = dict([(i,0) for i in "ACTG"])
for record in SeqIO.parse('../../../test/Homo_sapiens_3prime_UTR.fa', 'fasta'):
    for nucleotide in count_nucletides:
        count_nucletides[nucleotide] += record.seq.count(nucleotide)
print('\n'.join(['{}: {}'.format(i,count_nucletides[i]) for i in count_nucletides]))

A: 5919083
C: 4863209
T: 6414380
G: 4935864


In [9]:
s = sum(count_nucletides.values())
a = {k: v / s for k, v in count_nucletides.items()}

In [10]:
a

{'A': 0.26743808301046024,
 'C': 0.21973121381119634,
 'T': 0.2898167656883061,
 'G': 0.22301393749003728}

Example script usage ^^

In [ ]:
#!cd ML4RG-2023-project && python main.py --test --fasta ../Homo_sapiens_3prime_UTR.fa --species_list 240_species.txt --output_dir ./test --model_weight ../MLM_mammals_species_aware_5000_weights

In [9]:
file_path = 'test_df.pickle'
if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        train_df = pickle.load(f)
else:
    # load the fasta file and select the train data
    fasta_file = "../../../test/Homo_sapiens_3prime_UTR.fa"
    sequences = []
    for s in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(s.seq).upper())
    # get the train fraction
    val_fraction = 0.1
    N_train = int(len(sequences)*(1-val_fraction))
    test_data = sequences[N_train:]
    # store it as a dataframe
    test_df = pd.DataFrame({'3-UTR':test_data})
    with open(file_path, 'wb') as f:
        pickle.dump(test_df, f)
test_df

,3-UTR
0,CCCCCAGAACCAGTGGGACAAACTGCCTCCTGGAGGTTTTTAGAAA...
1,TATTGAGCCCTCAGAGAGTCCACAGTCCCTCCTCTCAGTTCAGTCT...
2,TATTCATTCCAACTGCTGCCCCTCTGTCTGCCTGGCTGAGATGCAT...
3,AACGGTGCGTTTGGCCAAAAAGAATCTGCATTTAGCACAAAAAAAA...
4,TAGTTTCTAACTGTCGGACCCGTCTGTAAACCAAGGACTATGAATA...
...,...
1809,AGCAAGCATTGAAAATAATAGTTATTGCATACCAATCCTTGTTTGC...
1810,AGCAAGCATTGAAAATAATAGTTATTGCATACCAATCCTTGTTTGC...
1811,GCCTACTTCATCTCAGGACCCGCCCAAGAGTGGCCGCGGCTTTGGG...
1812,TTGTCAGTCTGTCTGCTCAGGACACAAGAACTAAGGGGCAACAAAT...


In [10]:
file_path = 'train_df.pickle'
if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        train_df = pickle.load(f)
else:
    # load the fasta file and select the train data
    fasta_file = "../../../test/Homo_sapiens_3prime_UTR.fa"
    sequences = []
    for s in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(s.seq).upper())
    # get the train fraction
    val_fraction = 0.1
    N_train = int(len(sequences)*(1-val_fraction))
    train_data = sequences[:N_train]
    # store it as a dataframe
    train_df = pd.DataFrame({'3-UTR':train_data})
    with open(file_path, 'wb') as f:
        pickle.dump(train_df, f)
train_df

,3-UTR
0,ATCTTATATAACTGTGAGATTAATCTCAGATAATGACACAAAATAT...
1,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...
2,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...
3,CCCACCTACCACCAGAGGCCTGCAGCCTCCCACATGCCTTAAGGGG...
4,TGGCCGCGGTGAGGTGGGTTCTCAGGACCACCCTCGCCAAGCTCCA...
...,...
16315,CCGTATGAAGATGTCCTGTTAAATTTACAACACTAACGATGTAGAC...
16316,ACACACCCCCGAAAAACACAAGACCGACCCAAAATCTAGAGGAAAG...
16317,AGAAGCTAAAAGGAAAGAAAATAAATCTATCAAAATTACCCTAAAC...
16318,CTTCACTTTTGGGCTCAAGGACTGTGTGAACCAACAAGGGGCCAGT...


In [11]:
file_path = 'kmer_train.pickle'
if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        kmer_train = pickle.load(f)
else:
    # get the frequency counts of all motifs till 11mer
    kmer_train = KmerCountNew(2,pseudocount=0.1)
    kmer_train.compute_counts(train_df['3-UTR'])
    kmer_train.kmer_counts_dict

    # save dictionary pickle file
    with open('kmer_train.pickle', 'wb') as f:
        pickle.dump(kmer_train, f)

  0%|          | 0/16320 [00:00<?, ?it/s]

100%|██████████| 16320/16320 [01:05<00:00, 250.06it/s]


In [13]:
dinuc_dist = np.array([[[0.26743808301046024,0.21973121381119634, 0.22301393749003728, 0.2898167656883061],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.2968737832744875, 0.18507058520226632, 0.26581813454136444,0.26581813454136444],
        [0.32053421022722217, 0.2803858410399341, 0.0554500543745703, 0.34362989435827346],
        [0.26000757243673117, 0.2138329457921364, 0.26097487205728753, 0.24694501885793302],
        [0.20207800866762973, 0.20690293271757126, 0.27748861612369985, 0.3135304424910991]]])

In [14]:
chain = MarkovChainNew(kmer_train, dinuc_dist)

In [15]:
chain.markov_matrix

array([[[0.26743808, 0.21973121, 0.22301394, 0.28981677],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.29687378, 0.18507059, 0.26581813, 0.26581813],
        [0.32053421, 0.28038584, 0.05545005, 0.34362989],
        [0.26000757, 0.21383295, 0.26097487, 0.24694502],
        [0.20207801, 0.20690293, 0.27748862, 0.31353044]]])

In [16]:
chain.impute_for_seq("AAACT", 1)

array([[0.26743808, 0.21973121, 0.22301394, 0.28981677],
       [0.29687378, 0.18507059, 0.26581813, 0.26581813],
       [0.29687378, 0.18507059, 0.26581813, 0.26581813],
       [0.29687378, 0.18507059, 0.26581813, 0.26581813],
       [0.32053421, 0.28038584, 0.05545005, 0.34362989]])

In [17]:
markov_model = MarkovModelNew(
    kmer_train,
    markov_matrix_path="markov_model.npy",
    order=1,
    bidirectional=False,
    test_df_path='test_df.pickle',
    dinuc_dist = dinuc_dist)

In [18]:
markov_model.test()

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
!cp -r "/content/prbs.pt" "/content/drive/MyDrive/MLRG2023"

In [ ]:
class SeqDataset(Dataset):

    def __init__(self, fasta_fa, seq_df, transform):

        self.fasta = pysam.FastaFile(fasta_fa)

        self.seq_df = seq_df
        self.transform = transform



    def __len__(self):

        return len(self.seq_df)

    def __getitem__(self, idx):

        seq = self.fasta.fetch(self.seq_df.iloc[idx].seq_name).upper()
        #print(seq)

        species_label = self.seq_df.iloc[idx].species_label
        #print(species_label)
        motifs = {"GTATG":1}

        masked_sequence, target_labels_masked, target_labels, mask, _ = self.transform(seq, motifs = motifs)

        masked_sequence = (masked_sequence, species_label)

        return masked_sequence, target_labels_masked, target_labels

    def close(self):
        self.fasta.close()

# Read the data

In [ ]:
fasta_fa = "./Homo_sapiens_3prime_UTR.fa"
species_list = "ML4RG-2023-project/240_species.txt"

seq_df = pd.read_csv(fasta_fa + '.fai', header=None, sep='\t', usecols=[0], names=['seq_name'])
seq_df['species_name'] = seq_df.seq_name.apply(lambda x:x.split(':')[1])
species_encoding = pd.read_csv(species_list, header=None).squeeze().to_dict()
species_encoding = {species:idx for idx,species in species_encoding.items()}
species_encoding['Homo_sapiens'] = species_encoding['Pan_troglodytes']
seq_df['species_label'] = seq_df.species_name.map(species_encoding)
seq_df

In [ ]:
# Motif:id
motifs = {"GTATG":1}

In [ ]:
kseq_len = 5000
total_len = 5000

#
seq_transform = sequence_encoders.RollingMasker(mask_stride=50, frame=0)


test_dataset = SeqDataset(fasta_fa, seq_df, transform = seq_transform)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, num_workers = 1, collate_fn = None, shuffle = False)


# Load the model
## Model params

In [ ]:
gc.collect()
torch.cuda.empty_cache()
device = torch.device('cuda')
d_model = 128
n_layers = 4
dropout = 0.
learn_rate = 1e-4
weight_decay = 0.
output_dir = "./test/"
get_embeddings = True
save_at = []

## Model
If the following line fails:

```
model = model.to(device)
```
Either use:


```
device = torch.device("cpu")
```
Or go to Runtime -> change runtime type -> Hardware Accellerator 'GPU'



In [ ]:
species_encoder = SpecAdd(embed = True, encoder = 'label', d_model = d_model)

model = DSSResNetEmb(d_input = 5, d_output = 5, d_model = d_model, n_layers = n_layers, dropout = dropout, embed_before = True, species_encoder = species_encoder)

model = model.to(device)

model_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(model_params, lr = learn_rate, weight_decay = weight_decay)

last_epoch = 0
model_weight = "MLM_mammals_species_aware_5000_weights"
model.load_state_dict(torch.load(model_weight))

In [ ]:
predictions_dir = os.path.join(output_dir, 'predictions') #dir to save predictions
weights_dir = os.path.join(output_dir, 'weights') #dir to save model weights at save_at epochs
if save_at:
    os.makedirs(weights_dir, exist_ok = True)

def metrics_to_str(metrics):
    loss, total_acc, masked_acc = metrics
    return f'loss: {loss:.4}, total acc: {total_acc:.3f}, masked acc: {masked_acc:.3f}'

from helpers.misc import print    #print function that displays time
print(f'Test/Inference...')

test_metrics, test_embeddings =  train_eval.model_eval(model, optimizer, test_dataloader, device,
                                                          get_embeddings = get_embeddings, silent = True)




[2023/06/10-19:20:01]- Test/Inference...


/content/./ML4RG-2023-project/helpers/train_eval.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  species_label = torch.tensor(species_label).long().to(device)
/content/./ML4RG-2023-project/models/dss.py:335: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:276.)
  return einsum('chn,hnl->chl', W, S).float(), state                   # [C H L]
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dr